In [10]:
import sys, io, time, csv
import pandas as pd
from tqdm import tqdm
from selenium import webdriver
from selenium.common import exceptions

Extracts the comments from the Youtube video given by the URL.

In [13]:
def scrape(url):
    """
    Extracts the comments from the Youtube video given by the URL.
    Args:
        url (str): The URL to the Youtube video
    Raises:
        selenium.common.exceptions.NoSuchElementException:
        When certain elements to look for cannot be found
    """

    # Note: Download and replace argument with path to the driver executable.
    # Simply download the executable and move it into the webdrivers folder.
    driver = webdriver.Chrome('./chromedriver.exe')

    # Navigates to the URL, maximizes the current window, and
    # then suspends execution for (at least) 5 seconds (this
    # gives time for the page to load).
    driver.get(url)
    driver.maximize_window()
    time.sleep(10)

    try:
        # Extract the elements storing the video title and
        # comment section.
        title = driver.find_element_by_xpath('//*[@id="container"]/h1/yt-formatted-string').text
        comment_section = driver.find_element_by_xpath('//*[@id="comments"]')
    except exceptions.NoSuchElementException:
        # Note: Youtube may have changed their HTML layouts for
        # videos, so raise an error for sanity sake in case the
        # elements provided cannot be found anymore.
        error = "Error: Double check selector OR "
        error += "element may not yet be on the screen at the time of the find operation"
        print(error)

    # Scroll into view the comment section, then allow some time
    # for everything to be loaded as necessary.
    driver.execute_script("arguments[0].scrollIntoView();", comment_section)
    time.sleep(12)

    # Scroll all the way down to the bottom in order to get all the
    # elements loaded (since Youtube dynamically loads them).
    last_height = driver.execute_script("return document.documentElement.scrollHeight")

    while True:
        # Scroll down 'til "next load".
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")

        # Wait to load everything thus far.
        time.sleep(5)

        # Calculate new scroll height and compare with last scroll height.
        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # One last scroll just in case.
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")

    try:
        # Extract the elements storing the usernames and comments.
        username_elems = driver.find_elements_by_xpath('//*[@id="author-text"]')
        comment_elems = driver.find_elements_by_xpath('//*[@id="content-text"]')
    except exceptions.NoSuchElementException:
        error = "Error: Double check selector OR "
        error += "element may not yet be on the screen at the time of the find operation"
        print(error)

#     print("> VIDEO TITLE: " + title + "\n" + "====> " + str(len(username_elems)) + " Comments found. \n")

    with io.open('./data/results.csv', 'a', newline='', encoding="utf-16") as file:
        writer = csv.writer(file, delimiter =",", quoting=csv.QUOTE_ALL)
#         writer.writerow(["Username", "Comment"])
        for username, comment in zip(username_elems, comment_elems):
            writer.writerow([username.text, comment.text])

    driver.close()

In [14]:
scrape("https://www.youtube.com/watch?v=WuisRYDpQQg")

> VIDEO TITLE: حلقة بعنوان : بزاااف ديال البرد
====> 38 Comments found. 



In [26]:
yt_urls = pd.read_csv('./data/youtube_videos.csv')
yt_urls.head()

,url
0,https://www.youtube.com/watch?v=zpAxJfKxucw
1,https://www.youtube.com/watch?v=C5z27AuXRck
2,https://www.youtube.com/watch?v=61FilFpMVJ8
3,https://www.youtube.com/watch?v=Kxq1id9Be0o
4,https://www.youtube.com/watch?v=AEOFY1E499o


In [27]:
for url in tqdm(yt_urls["url"]):
    scrape(url)

  0%|                                                                                           | 0/51 [00:00<?, ?it/s]

> VIDEO TITLE: من طنجة..نايضة بالمحكمة وقاتل الطفل عدنان يعترف للقاضي بتفاصيل خطيرة..شوفو كيفاش قتل الضحية
====> 20 Comments found. 



  4%|███▎                                                                               | 2/51 [01:07<28:58, 35.48s/it]

> VIDEO TITLE: الفنان ولد الحوات فركع الرمانة وفضح المنادجر ديالو والقضية فيها الاعتداء على الشيخات والفلوس
====> 0 Comments found. 

> VIDEO TITLE: من الدارالبيضاء..فضيحة كبرى للفنان سعيد ولد الحوات يكشفها مدير أعماله: هاشنو دار ليا أسرار خطيرة
====> 60 Comments found. 



  6%|████▉                                                                              | 3/51 [01:51<30:24, 38.02s/it]

...

 96%|███████████████████████████████████████████████████████████████████████████▉   | 49/51 [4:44:13<15:24, 462.49s/it]

> VIDEO TITLE: الفيديو القنبلة..رقص مثير للشيخة الطراكس في حفل زفاف بنت الستاتي وهاشنو دارت سعيدة شرف
====> 721 Comments found. 



 98%|█████████████████████████████████████████████████████████████████████████████▍ | 50/51 [4:48:10<06:34, 394.82s/it]

> VIDEO TITLE: MANAL - NIYA (official parody by matrixmen )
====> 1540 Comments found. 



100%|███████████████████████████████████████████████████████████████████████████████| 51/51 [5:03:41<00:00, 357.28s/it]


In [24]:
with io.open('./data/results.csv', 'a', newline='', encoding="utf-16") as file:
        writer = csv.writer(file, delimiter =",", quoting=csv.QUOTE_ALL)
        writer.writerow(["Username", "Comment"])